# Bring Your Own Model を SageMaker で hosting する
* 先程学習したモデルを Notebook インスタンスで読み込み、改めて tensorflow で save し、自分で作成したモデルとする
* 自作モデルを SageMaker で hosting & 推論する

## 処理概要
* 先程学習したモデルを Notebook インスタンスにダウンロード
* TensorFlow で読み込み、推論し、改めて保存し直す
* 保存しなおしたモデルを S3 にアップロードする
* モデルを hosting する


In [ ]:
# notebook のセルの横方向の表示範囲を広げる
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sagemaker, yaml, tarfile, boto3, os, json, boto3
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
from sagemaker.tensorflow import TensorFlowModel
print(f'Current tensorflow Version ={tf.__version__}')

## 設定読み込み

In [ ]:
with open('./setting.yaml', 'r') as yml:
    config = yaml.load(yml)
best_model_uri = config['best_model_uri']
name = config['name']
timestamp = config['timestamp']
print(best_model_uri)

## モデルを Notebook インスタンスにダウンロード

In [ ]:
sagemaker.s3.S3Downloader.download(
    s3_uri=best_model_uri,
    local_path='./model/'
)

In [ ]:
# tar.gz の解凍
with tarfile.open('./model/model.tar.gz') as tar:
    tar.extractall('./model/')

## モデルの読み込み
* SageMaker Training で学習したものは当然ながら TensorFlow で学習したものと同一
* load してそのまま使えるかを確認する

In [ ]:
model = tf.keras.models.load_model('./model/000000001/')
model.summary()

In [ ]:
test_x = np.load('./test_x.npy')
plt.imshow(test_x[0,:,:,0],'gray')

In [ ]:
np.argmax(model.predict(test_x[0:1,:,:,:]))

## ダウンロードしたモデルを削除し、改めて保存しなおす
* tensorflow の save を利用して保存した後、tar.gz に圧縮する

In [ ]:
!rm -r ./model

In [ ]:
model_dir = './000000002'
tar_name = os.path.join(model_dir, 'model.tar.gz')

In [ ]:
model.save(model_dir)
with tarfile.open(tar_name, mode='w:gz') as tar:
    tar.add(model_dir)

## tar.gz に圧縮したモデルを S3 にアップロードする

In [ ]:
sess = sagemaker.session.Session()
bucket = sess.default_bucket()
model_s3_path = f's3://{bucket}/{name}-model-{timestamp}'
print(model_s3_path)

In [ ]:
model_s3_uri = sagemaker.s3.S3Uploader.upload(
    local_path = tar_name,
    desired_s3_uri = model_s3_path
)
print(model_s3_uri)

## S3 にアップロードしたモデルを SageMaker 管理のモデルとして登録する
* TensorFlow で作成したモデルは [TensorFlowModel](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html?highlight=TensorFlowModel#sagemaker.tensorflow.model.TensorFlowModel) で読み込む
* 推論に利用する TensorFlow のコンテナイメージを [sagemaker.image_uris.retrieve](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html?highlight=sagemaker.image_uris.retrieve#sagemaker.image_uris.retrieve) で事前に取得しておく(SageMaker にこのモデルは TensorFlow の 2.1 で作られたものであることを報せる必要がある）

In [ ]:
container_image_uri = sagemaker.image_uris.retrieve(
    "tensorflow", 
    boto3.Session().region_name, 
    version='2.1',
    instance_type = 'ml.m5.large',
    image_scope = 'inference'
)

In [ ]:
tf_model = TensorFlowModel(
    model_data=model_s3_uri,
    role=sagemaker.get_execution_role(),
    image_uri = container_image_uri,
)

## hosting して推論の確認を行う

In [ ]:
predictor = tf_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
)

In [ ]:
np.argmax(predictor.predict(test_x[0:1,:,:,:])['predictions'])

## 推論のみを行う場合
* 毎回モデルを読み込んで推論するわけではなく、ほとんどのケースでは モデルを hosting している endpoint で推論のみを行う
* 今回は sagemaker sdk を利用して推論のみをする場合と、boto3 を用いて推論する場合の 2 種類を試す
* 推論をするのに必要な情報は endpoint_name と推論するデータのみ

In [ ]:
# endpoint_name を予め取得する (マネジメントコンソールでも確認可能)
endpoint_name = predictor.endpoint_name
print(endpoint_name)

### sagemaker SDK を利用する場合

In [ ]:
predictor2 = sagemaker.predictor.Predictor(
    endpoint_name,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
np.argmax(predictor2.predict(test_x[0:1,:,:,:])['predictions'])

### boto3 を利用する場合

In [ ]:
client = boto3.client('sagemaker-runtime')

In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps({"instances": test_x[0:1,:,:,:].tolist()}),
    ContentType='application/json'
)
np.argmax(np.array(json.load(response['Body'])['predictions'][0]))